In [ ]:
!pip install crossrefapi
!pip install orcid

  Using cached orcid-1.0.3.tar.gz (10 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for orcid: filename=orcid-1.0.3-py3-none-any.whl size=10443 sha256=abca2eb2f57617ad612d26060a5ea2d749b9039fb9613bedd90be3cf34a35163
  Stored in directory: /root/.cache/pip/wheels/77/97/ed/c29a05c6a5bfde56e7adbfa37dea18a6bb08a27241ba0a7dcc
Successfully built orcid


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Journal tuples

In [ ]:
# helpers to serialize the text
import math

def sql_literal(value):
    if value is None:
        return "NULL"
    if isinstance(value, float) and math.isnan(value):
        return "NULL"
    if isinstance(value, str):
        return "'" + value.replace("'", "''") + "'"
    return str(value)

import re

def normalize(s: str) -> str:
    if s is None:
        return s

    s = (
        s.replace("’", "'")      # curly single quote
         .replace("‘", "'")      # curly single quote
         .replace("‐", "-")      # unicode hyphen
         .replace("–", "-")      # en dash (U+2013)
         .replace("—", "-")      # em dash (U+2014)
    )

    # collapse multiple spaces
    s = re.sub(r"\s+", " ", s)

    return s.strip()


In [ ]:
# Journal files pulled as CSV and converted to tsv from here: https://jcr-clarivate-com.proxy1.library.jhu.edu/jcr/browse-journals
journal_list="/content/drive/MyDrive/databases_project/journal_impact_factor.tsv"
top_10={}
with open(journal_list, 'r') as file:
    # Skip first header lines
    for i in range(0,2):
        file.readline()
    columns=[item.strip().strip('"') for item in file.readline().strip().strip('"').split("\t")]
    for n in range(0,10):
        journal=[item.strip().strip('"') for item in file.readline().strip().strip('"').split("\t")]
        top_10[journal[0]]=dict(zip(columns[1:], journal[1:]))

In [ ]:
journal_issns=[]
with open("journal_tuples.txt", "w") as j:
    for journal_name in top_10.keys():
        journal_values = [journal_name] # Add the full journal name (key of top_10) once
        # Collect values for the current journal from its inner dictionary, excluding the 'Journal name' entry
        for key in top_10[journal_name].keys():
            if key != 'Journal name': # This key holds the abbreviated name, causing the duplication
                journal_values.append(top_10[journal_name][key])
        # Convert the list of strings to a tuple, then to its string representation, and add a newline
        values_sql = "(" + ", ".join(sql_literal(v) for v in journal_values) + ")"
        j.write(
            f"INSERT INTO Journals VALUES {values_sql} "
            "ON CONFLICT (LongName) DO NOTHING;\n"
        )

In [ ]:
publishers = set()
for journal_name in top_10.keys():
    publishers.add(top_10[journal_name]["Publisher"])

# Article Tuples

In [ ]:
import datetime
from crossref.restful import Works

works = Works() # Initialize Works object

# Set the start date to January 1, 2025

articles = []
years = [2023, 2024, 2025]
query_limit = 1000

for publisher in publishers:
    for year in years:

        from_date = datetime.date(year, 1, 1)
        from_date_str = from_date.strftime('%Y-%m-%d')
        until_date_str = datetime.date.today().strftime('%Y-%m-%d')
        print(f"Searching for articles published by {publisher} from {year}. (limit {query_limit} per query)...")
        publisher_articles = []
        article_iterator = works.query(
            publisher_name=publisher
        ).filter(
            from_pub_date=from_date_str,
            until_pub_date=until_date_str
        )

        for article in article_iterator:
            publisher_articles.append(article)
            if len(publisher_articles) >= query_limit: # Manually enforce the limit per publisher
                break

        articles.extend(publisher_articles) # Add collected articles to the main list

Searching for articles published by WILEY from 2023. (limit 1000 per query)...
Searching for articles published by WILEY from 2024. (limit 1000 per query)...
Searching for articles published by WILEY from 2025. (limit 1000 per query)...
Searching for articles published by ELSEVIER SCIENCE INC from 2023. (limit 1000 per query)...
Searching for articles published by ELSEVIER SCIENCE INC from 2024. (limit 1000 per query)...
Searching for articles published by ELSEVIER SCIENCE INC from 2025. (limit 1000 per query)...
Searching for articles published by NATURE PORTFOLIO from 2023. (limit 1000 per query)...
Searching for articles published by NATURE PORTFOLIO from 2024. (limit 1000 per query)...
Searching for articles published by NATURE PORTFOLIO from 2025. (limit 1000 per query)...


In [ ]:
articles_tuples = []
breaking = 0
for article in articles:
    # Skip cases where there is no article title
    title = article.get('title', None)
    if title is None:
        continue

    # Extract authors
    # Skip articles with no authors listed
    authors = article.get('author', [])
    author_names = []
    if authors:
        for author in authors:
            given_name = author.get('given', '')
            family_name = author.get('family', '')
            if given_name or family_name:
                author_names.append(f"{given_name} {family_name}".strip())
    else:
        continue

    author_number = 1
    for author in author_names:
        current_article = []

        # FIX: use the full title, not title[0]
        current_article.append(article.get('title', 'No Title Provided'))

        current_article.append(author)
        current_article.append(author_number)
        author_number += 1

        current_article.append(article.get('DOI', None))
        current_article.append(article.get('publisher', None))

        # FIX: round score to match DECIMAL(5,1)
        score = article.get('score', None)
        if score is not None:
            try:
                score = round(float(score), 1)
            except:
                score = None
        current_article.append(score)

        current_article.append(article.get('URL', None))

        # Safely get 'published' and then 'date-parts'
        published_info = article.get('published', None)
        if published_info != None:
            try:
                current_article.append(str(published_info['date-parts'][0][0]))  # year
            except:
                current_article.append(None)
        else:
            current_article.append(None)

        ISSN = article.get('ISSN', None)
        if ISSN != None:
            current_article.append(ISSN[0])
        else:
            current_article.append(None)
        journal=article.get('container-title', None)
        if journal != None:
            current_article.append(journal[0])
        ref_count = article.get("reference-count", None)
        if ref_count is None:
            ref_count = article.get("references-count", None)

        current_article.append(ref_count)
        current_article.append(article.get('is-referenced-by-count', None))

        articles_tuples.append(tuple(current_article))

In [ ]:
articles[1]

{'indexed': {'date-parts': [[2025, 3, 14]],
  'date-time': '2025-03-14T04:29:29Z',
  'timestamp': 1741926569921,
  'version': '3.38.0'},
 'reference-count': 23,
 'publisher': 'Wiley',
 'issue': '185',
 'license': [{'start': {'date-parts': [[2025, 2, 24]],
    'date-time': '2025-02-24T00:00:00Z',
    'timestamp': 1740355200000},
   'content-version': 'vor',
   'delay-in-days': 0,
   'URL': 'http://creativecommons.org/licenses/by-nc/4.0/'}],
 'content-domain': {'domain': ['onlinelibrary.wiley.com'],
  'crossmark-restriction': True},
 'short-container-title': ['New Drctns Student Lead'],
 'published-print': {'date-parts': [[2025, 3]]},
 'abstract': '<jats:title>ABSTRACT</jats:title><jats:p>This article applies the critical media concept of organic representation to leadership studies as an analytic of how various creators in popular culture today are not just writing inclusive storytelling but, more notably, modeling new modes of production and self‐presentation that are actively challeng

In [ ]:
with open("article_tuples.txt", "w", encoding="utf-8") as j:
    for article in articles_tuples:
        fixed = list(article)
        if isinstance(fixed[0], list):
            fixed[0] = fixed[0][0]

        cleaned = [
            normalize(v) if isinstance(v, str) else v
            for v in fixed
        ]

        values_sql = ", ".join(sql_literal(v) for v in cleaned)

        j.write(
            f"INSERT INTO Articles VALUES ({values_sql}) "
            f"ON CONFLICT (Title, Author_Number) DO NOTHING;\n"
        )

# ROR tuples

In [ ]:
affiliation_list = set()
for article in articles:
    authors = article.get('author', [])
    if authors:
        for author in authors:
            affiliation = author.get('affiliation', "")
            if affiliation:
                for n in affiliation:
                    affiliation_list.add(n['name'])

In [ ]:
import requests
institution_details = []
weird_format= []

for affiliation in affiliation_list:
    url = f"https://api.ror.org/organizations?query={affiliation}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()

        if 'items' in data and isinstance(data['items'], list) and len(data['items']) > 0:
            try:
                item = data['items'][0]
                name = item.get('name', affiliation) # Use affiliation as fallback for name
                established = item.get('established', None)

                country_code = None
                locations = item.get('locations')

                if locations and isinstance(locations, list) and len(locations) > 0:
                    first_location = locations[0]
                    geonames_details = first_location.get('geonames_details')
                    if geonames_details:
                        country_code = geonames_details.get('country_code', None)
                institution_details.append([name, established, country_code])

            except (IndexError, KeyError, TypeError) as e:
                weird_format.append(affiliation)
            except Exception as e:
                weird_format.append(affiliation)
        else:
            weird_format.append(affiliation)
    else:
        weird_format.append(affiliation)

KeyboardInterrupt: 

In [ ]:
institution_details[0]

['Institute of Communications and Navigation German Aerospace Center (DLR e.V.)  Wessling Germany',
 1969,
 'DE']

In [ ]:
with open("institution_tuples.txt", "w") as j:
    for institution in institution_details:
        values_sql = ", ".join(sql_literal(v) for v in institution)
        j.write(
            f"INSERT INTO Institutions VALUES ({values_sql}) "
            "ON CONFLICT (Name) DO NOTHING;\n"
        )

# Authors

In [ ]:
from google.colab import userdata
ORCID_client_id=userdata.get("ORCID_client_id")
ORCID_client_secret=userdata.get("ORCID_client_secret")

In [ ]:
import orcid

client = orcid.PublicAPI(ORCID_client_id, ORCID_client_secret)
consecutive_api_failures = 0
authors_processed = {}
author_orcids_found = {}

print("ORCID API client initialized and variables set.")

ORCID API client initialized and variables set.


In [ ]:
unique_authors = {}

for article in articles:
    authors = article.get('author', [])
    if not authors:
        continue

    for author in authors:
        given_name = author.get('given', '')
        family_name = author.get('family', '')

        # Construct full_name for easier identification, though key is (family, given)
        full_name = f"{given_name} {family_name}".strip()

        # Extract affiliation names
        affiliations = author.get('affiliation', [])
        affiliation_names = set()
        if affiliations:
            for aff_item in affiliations:
                if 'name' in aff_item:
                    affiliation_names.add(aff_item['name'])
        else:
            # If no specific affiliations are provided, use a placeholder
            affiliation_names.add('No Affiliation Provided')

        author_key = (family_name, given_name) # Using a tuple as key for uniqueness

        if author_key not in unique_authors:
            unique_authors[author_key] = set()
        unique_authors[author_key].update(affiliation_names)

print(f"Processed {len(articles)} articles. Found {len(unique_authors)} unique authors with affiliations.")

Processed 9000 articles. Found 18128 unique authors with affiliations.


In [ ]:
author_queries = {}

for (family_name, given_name), affiliations in unique_authors.items():
    queries = []

    primary_affiliation = 'No Affiliation Provided'
    if affiliations and 'No Affiliation Provided' not in affiliations: # Prioritize actual affiliations over placeholder
        primary_affiliation = list(affiliations)[0]

    # Query 1: Full name and primary affiliation (if available and not placeholder)
    if given_name and family_name and primary_affiliation != 'No Affiliation Provided':
        queries.append(f"family-name:{family_name} AND given-names:{given_name} AND affiliation-org-name:\"{primary_affiliation}\"")

    # Query 2: Full name only
    if given_name and family_name:
        queries.append(f"family-name:{family_name} AND given-names:{given_name}")

    # Query 3: Family name only (if given name is empty or affiliation not used)
    if family_name and not given_name and not queries: # Only add if no other query was possible yet with the family name
        queries.append(f"family-name:{family_name}")
    elif family_name and given_name and not queries[0].startswith(f"family-name:{family_name} AND given-names:{given_name}"): # If full name query exists, skip family-name only
        queries.append(f"family-name:{family_name}")

    # Query 4: Given name only (as a last resort if family name is also missing)
    if given_name and not family_name and not queries:
        queries.append(f"given-names:{given_name}")

    author_queries[(family_name, given_name)] = queries

In [ ]:
orcid_tuples = []
MAX_CONSECUTIVE_API_FAILURES = 10

try:
    public_access_token = client.get_search_token_from_orcid()
    print("Successfully obtained public access token for ORCID API.")
except Exception as e:
    print(f"Error obtaining public access token: {e}")
    public_access_token = None # Ensure it's None if acquisition fails

if public_access_token is None:
    print("Cannot proceed with ORCID searches without a public access token.")
else:
    consecutive_api_failures = 0
    author_orcids_found = {}

    for author_key, queries in author_queries.items():
        if author_key in author_orcids_found:
            continue # Skip if ORCID already found for this author

        found_orcid_id = None
        current_primary_affiliation = list(unique_authors[author_key])[0] if unique_authors[author_key] else 'No Affiliation Provided'

        for query in queries:
            if consecutive_api_failures >= MAX_CONSECUTIVE_API_FAILURES:
                print(f"Stopping ORCID search due to {MAX_CONSECUTIVE_API_FAILURES} consecutive API failures.")
                break # Break inner loop

            try:
                search_response = client.search(query, access_token=public_access_token)

                if search_response and 'result' in search_response:
                    consecutive_api_failures = 0 # Reset failure counter on success
                    search_results = search_response['result']

                    if search_results:
                        best_match_orcid = None
                        # Prioritize by affiliation if available and not 'No Affiliation Provided'
                        if current_primary_affiliation != 'No Affiliation Provided':
                            for result in search_results:
                                # Simplified approach: if a query with affiliation yielded results, and there's a single result, take it.
                                # This logic is based on the assumption that if an affiliation was used in the query, and it returned a single result,
                                # that result is the most relevant. If multiple results or a broader query, take the first.
                                if 'affiliation-org-name' in query and len(search_results) == 1:
                                    best_match_orcid = result['orcid-identifier']['path']
                                    break # Found a highly specific match

                            if best_match_orcid is None: # If no single specific match by affiliation query, take first
                                best_match_orcid = search_results[0]['orcid-identifier']['path']
                        else: # No primary affiliation to match, just take the first result
                            best_match_orcid = search_results[0]['orcid-identifier']['path']

                        found_orcid_id = best_match_orcid
                        break # ORCID found, break from inner query loop
                else:
                    # Search response was empty or did not contain 'result'
                    consecutive_api_failures += 1

            except Exception as e:
                consecutive_api_failures += 1
                print(f"Warning: ORCID API call failed for query '{query}': {e}. Consecutive failures: {consecutive_api_failures}")
                if consecutive_api_failures >= MAX_CONSECUTIVE_API_FAILURES:
                    break # Break inner loop if too many failures

        if found_orcid_id:
            author_orcids_found[author_key] = found_orcid_id
            family_name_str, given_name_str = author_key
            author_display_name = f"{given_name_str} {family_name_str}".strip()
            orcid_tuples.append((author_display_name, current_primary_affiliation, found_orcid_id))
        else:
            family_name_str, given_name_str = author_key
            author_display_name = f"{given_name_str} {family_name_str}".strip()

        if consecutive_api_failures >= MAX_CONSECUTIVE_API_FAILURES:
            break # Break outer loop as well

print(f"Found {len(orcid_tuples)} ORCID records.")
print(f"Example ORCID tuples: {orcid_tuples[:5]}")

Successfully obtained public access token for ORCID API.
Found 13483 ORCID records.
Example ORCID tuples: [('Miro Demol', 'No Affiliation Provided', '0000-0002-5492-2874'), ('Naikoa Aguilar-Amuchastegui', 'No Affiliation Provided', '0000-0002-5072-0079'), ('Gabija Bernotaite', 'No Affiliation Provided', '0009-0004-5550-1109'), ('Mathias Disney', 'No Affiliation Provided', '0000-0002-2407-4026'), ('Laura Duncanson', 'No Affiliation Provided', '0000-0003-4031-3493')]


In [ ]:
with open("orcid_tuples.txt", "w", encoding="utf8") as g:
  for author in orcid_tuples:
    cleaned = [normalize(v) for v in author]
    values_sql = ", ".join(sql_literal(v) for v in cleaned)

    g.write(
      f"INSERT INTO Author VALUES ({values_sql}) "
      "ON CONFLICT (Name) DO NOTHING;\n"
    )


# Race tuples

In [ ]:
#Race by name data found here https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/SGKW0K
with open("race_tuples.txt", "w") as g:
    with open("/content/drive/MyDrive/databases_project/last_nameRaceProbs.tab", 'r') as f:
        race_list = f.readline().strip().split("\t")[1:]
        for line in f:
            line = line.strip().split("\t")
            values_to_check = line[1:]
            max_value_str = max(values_to_check)
            race = race_list[values_to_check.index(max_value_str)]
            name_literal = sql_literal(line[0].strip('"'))
            race_literal = sql_literal(race)

            g.write(
                f"INSERT INTO Race VALUES ({name_literal}, {race_literal}) "
                f"ON CONFLICT (Name) DO NOTHING;\n"
            )

In [ ]:
# not enough race values attached to the names so gonna augment with a kaggle dataset
with open("race_tuples.txt", "a") as g:
    with open("/content/drive/MyDrive/databases_project/names-by-nationality.csv", 'r') as f:
        next(f)
        for line in f:
            line = line.strip().split(",")
            name = sql_literal(line[0])
            race = sql_literal(line[2])
            g.write(
                f"INSERT INTO Race VALUES ({name}, {race}) "
                f"ON CONFLICT (Name) DO NOTHING;\n"
            )



# Gender Tuples

In [11]:
author_names=set()
with open("/content/drive/MyDrive/databases_project/orcid_tuples.sql") as f:
    for line in f:
        line = str(line.split(",")[0].split("(")[1].strip("'").split(" ")[0]).lower()
        author_names.add(line)

In [12]:
author_names

{'zaka',
 'marcus',
 'remigio',
 'gunter',
 'sy',
 'matt',
 'xing-yuan',
 'xiang',
 'anselm',
 'şule',
 'menghua',
 'rupal',
 'wim',
 'jens-christian',
 'yan-gui',
 'xiaozhen',
 'usame',
 'quan',
 'rajkumar',
 'yujia',
 'haiyun',
 'kashif',
 'debotra',
 'yu-ting',
 'mo-kyung',
 'haonan',
 'shita',
 'giovanni',
 'sai',
 'duoqian',
 'zhaoyong',
 'desi',
 'danyang',
 'chamila',
 'mao',
 'nouamane',
 'ajit',
 'maksym',
 'ukadike',
 'abooali',
 'chase',
 'yiyang',
 'xinyu',
 'yohei',
 'catia',
 'mariko',
 'aurélie',
 'supreet',
 'iêda',
 'jihong',
 'karlijn',
 'orlando',
 'swetha',
 'spiros',
 'ouhara',
 'sharma',
 'soumyodeep',
 'minyi',
 'sato',
 'yangyang',
 'sen',
 'dehui',
 'cheikh',
 'juliana',
 'yeonjae',
 'rita',
 'saba',
 'javad',
 'shouhong',
 'gharib',
 'longjun',
 'christine',
 'leona',
 'leigh',
 'zhenmei',
 'amogh',
 'hikmet',
 'siri',
 'ermanno',
 'kathleen',
 'samira',
 'yao-yue',
 'itika',
 'adile',
 'xifan',
 'tadaaki',
 'golshan',
 'nathalie',
 'haichao',
 'venla',
 'fuch

In [13]:
# data from: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/MSEGSJ
gender_data = "/content/drive/MyDrive/databases_project/gender_name_code.tsv"
gender_dict = {}
with open(gender_data, "r", encoding="utf-8") as f:

    next(f)  # skip header

    for line in f:
        parts = line.strip().split("\t")

        name = parts[0].strip().replace("'", "").replace('"', '').replace('\\', '')
        first_name = name.split(" ")[0]
        if not first_name in author_names:
            continue
        country = parts[1].strip().replace("\"", "")
        gender = parts[2].strip().replace("\"", "")
        if gender == "?":
            gender = "A"
        prob = float(parts[3].strip())
        #new name
        if name not in gender_dict.keys():
            gender_dict[name] = {}
            gender_dict[name][country] = [prob, gender]
        # New country for name
        elif country not in gender_dict[name].keys():
            gender_dict[name][country] = [prob, gender]
        # Higher prob for existing country
        else:
            if prob > gender_dict[name][country][0]:
                gender_dict[name][country] = [prob, gender]
            elif prob == gender_dict[name][country][0] and (prob == 0.5):
                gender_dict[name][country] = [1, "A"]

In [ ]:
gender_dict.keys()

dict_keys(['s', 'a', 'a a', 'a aff', 'a ai', 'a an', 'a ang', 'a aron', 'a aya', 'a ba', 'a bai', 'a ban', 'a bang', 'a bao', 'a bei', 'a ben', 'a bi', 'a bian', 'a biao', 'a bin', 'a bing', 'a bo', 'a bu', 'a cai', 'a cang', 'a cao', 'a ce', 'a cen', 'a ceng', 'a cha', 'a chai', 'a chan', 'a chang', 'a chao', 'a chen', 'a cheng', 'a chi', 'a chong', 'a chou', 'a chu', 'a chuan', 'a chuang', 'a chun', 'a ci', 'a cong', 'a cui', 'a cun', 'a da', 'a dan', 'a dang', 'a dao', 'a de', 'a dele', 'a di', 'a dian', 'a diao', 'a die', 'a ding', 'a dong', 'a dou', 'a du', 'a duan', 'a dui', 'a dun', 'a duo', 'a e', 'a en', 'a er', 'a fa', 'a fan', 'a fang', 'a fei', 'a fen', 'a feng', 'a fu', 'a g', 'a ga', 'a gai', 'a gan', 'a gang', 'a gao', 'a ge', 'a gen', 'a geng', 'a gou', 'a gu', 'a guai', 'a guan', 'a guang', 'a gui', 'a gun', 'a guo', 'a hai', 'a han', 'a hannan', 'a hao', 'a he', 'a hei', 'a heng', 'a hong', 'a hou', 'a hu', 'a hua', 'a huai', 'a huan', 'a huang', 'a hui', 'a huo', 'a 

In [14]:
with open("Gender_tuples.txt", "w") as g:
    for key in gender_dict.keys():
        for country in gender_dict[key].keys():
            name_sql = sql_literal(key)
            country_sql = sql_literal(country)
            gender_sql = sql_literal(gender_dict[key][country][1])
            g.write(
                f"INSERT INTO Gender VALUES ({name_sql}, {country_sql}, {gender_sql});\n"
            )

In [ ]:
# author_fullnames = set()

# for last, first in unique_authors:  # authors = your dict_keys(...)
#     if first:  # case: last name in first position, first name in second
#         full = f"{first.strip()} {last.strip()}"
#     else:      # case: first arg is already a full name
#         full = last.strip()

#     # Clean formatting
#     full = full.replace("  ", " ").strip()
#     author_fullnames.add(full)

# cleaned = {n for n in author_fullnames if n.strip()}
# print(cleaned)

{'Yanpei Wu', 'Jacek Banasiak', 'Wu Patimanabudu', 'Paola Galozzi', 'Hamida Seba', 'Yingyu Liu', 'Ralf Greiner', 'Kocherlakota Snigdha', 'Amit Agarwal', 'Linan Ye', 'Yang Zhiqing', 'Enbo Liu', 'Jennifer L. Bailit', 'Sheila Cyril', 'Wen-Ming Cao', 'Alireza Fereydooni', 'Muhammad Izzat Samir Abdul Jalil', 'Gillian Wylie', 'Tarun Goma', 'Fatemeh Karami Zarandi', 'Shinta Saito', 'Tania Serna', 'Nayia Cominos', 'Xinhui Quan', 'Ayşe İrem Özmen', 'Julio Ortega-Usobiaga', 'Angela Starkweather', 'Anthony Breitbach', 'Magda Mendez', 'Fangjian Zhu', 'N. Bolaños', 'S. C. Vetrivel', 'Jeferson Sandoval', 'Boto Simatupang', 'Lai Hui Charmaine Lee', 'Helen N Schwerdt', 'Hong J. Kim', 'Rebecca J. Post', 'Donghong Ning', 'Emily Weisberg', 'Zayira G. Alváez', 'Paulo Novais', 'Liang Hui', 'Anda M. Naciu', 'Rafal Madonski', 'Wan Nordin Wan Hussin', 'Yue Pan', 'Jianlun Xu', 'D. Charalambidis', 'Guixiu Shi', 'Elizabeth Tong', 'Y. Shao', 'Jin Fang', 'Thomas Klinger', 'Mei Wang', 'Pedro Pinto Santos', 'J. C. J

In [ ]:
# gender_data = "/content/drive/MyDrive/databases_project/gender_name_code.tsv"
# gender_tuples = "gender_tuples.txt"
# names = set()

# with open(gender_data, "r", encoding="utf-8") as f, \
#      open(gender_tuples, "w", encoding="utf-8") as g:

#     next(f)  # skip header

#     for line in f:
#         parts = line.strip().split("\t")

#         name = parts[0].strip().replace("'", "").replace('"', '').replace('\\', '')
#         country = parts[1].strip().replace("\"", "")
#         gender = parts[2].strip().replace("\"", "")
#         names.add(name)

#         name_sql = sql_literal(name)
#         country_sql = sql_literal(country)
#         gender_sql = sql_literal(gender)

#         g.write(
#             f"INSERT INTO Gender VALUES ({name_sql}, {country_sql}, {gender_sql}) "
#             f"ON CONFLICT (Name) DO NOTHING;\n"
#         )

# print(names)

Buffered data was truncated after reaching the output size limit.

In [ ]:
# # kaggle dataset with gender information

# gender_data = "/content/drive/MyDrive/databases_project/gender.csv"
# gender_tuples = "gender_tuples.txt"
# with open(gender_data, "r", encoding="utf-8") as f, \
#   open(gender_tuples, "w", encoding="utf-8") as g:
#   next(f)
#   for line in f:
#     parts = line.strip().split(",")
#     name = parts[0].strip()
#     gender = parts[1].strip()

#     name_sql = sql_literal(name)
#     gender_sql = sql_literal(gender)
#     g.write(
#         f"INSERT INTO Gender VALUES ({name_sql}, {gender_sql}) "
#         f"ON CONFLICT (Name) DO NOTHING;\n"
#     )

# Country Code Tuples

In [ ]:
#Country code data
import kagglehub
import csv

data = kagglehub.dataset_download("juanumusic/countries-iso-codes")
data = data+"/wikipedia-iso-country-codes.csv"
with open("country_tuples.txt", "w") as g:
    with open(data, "r") as f:
        next(f)  # skip header
        reader = csv.reader(f)
        for codes in reader:
            codes = [codes[0], codes[1]]   # swapped the ordering (name, code)

            values_sql = ", ".join(sql_literal(v) for v in codes)
            g.write(
                f"INSERT INTO Countries VALUES ({values_sql}) "
                "ON CONFLICT (CountryCode) DO NOTHING;\n"
            )

100%|██████████| 4.16k/4.16k [00:00<00:00, 9.03MB/s]

Extracting files...


# DEFINE TABLES

In [10]:
institutions_table="""
CREATE TABLE IF NOT EXISTS Institutions (
    Name            VARCHAR(1000) PRIMARY KEY,
    Established     INTEGER,
    CountryCode     CHAR(2)
);

-- allows No affiliation provided as an option
INSERT INTO Institutions (Name, Established, CountryCode)
VALUES ('No Affiliation Provided', NULL, NULL)
ON CONFLICT (Name) DO NOTHING;
"""

author_table = """
CREATE TABLE IF NOT EXISTS Author (
    Name            VARCHAR(100) PRIMARY KEY,
    Affiliation     VARCHAR(1000),
    ORCID           CHAR(19),
    FOREIGN KEY (Affiliation) REFERENCES Institutions(Name)
);
"""

race_table= """
DROP TABLE Race;
CREATE TABLE IF NOT EXISTS Race (
    Name            VARCHAR(100) PRIMARY KEY,
    RaceLabel       VARCHAR(20),
    FOREIGN KEY (Name) REFERENCES Author(Name)
);
"""

journals_table="""
CREATE TABLE IF NOT EXISTS Journals (
    LongName        VARCHAR(100) PRIMARY KEY,
    ShortName       VARCHAR(100),
    Publisher       VARCHAR(100),
    ISSN            VARCHAR(10),
    eISSN           VARCHAR(10),
    Category        VARCHAR(50),
    Edition         VARCHAR(10),
    ImpactFactor    DECIMAL(5,1)
);
"""

articles_table="""
DROP TABLE Articles;
CREATE TABLE IF NOT EXISTS Articles (
    Title                   VARCHAR(300),
    Author                  VARCHAR(100),
    Author_Number           INTEGER,
    DOI                     VARCHAR(200),
    Publisher               VARCHAR(200),
    Score                   DECIMAL(5,1),
    URL                     VARCHAR(500),
    Published               VARCHAR(100),
    ISSN                    VARCHAR(20),
    Journal                 VARCHAR(1000),  -- jounral name
    References_Count        INTEGER,
    Is_Referenced_By_Count  INTEGER,
    FOREIGN KEY (Author) REFERENCES Author(Name),
    PRIMARY KEY (Title, Author_Number)
);
"""

country_table="""
CREATE TABLE IF NOT EXISTS Countries (
    Name            VARCHAR(100),
    CountryCode     CHAR(2) PRIMARY KEY
);
"""

gender_table="""
DROP TABLE Gender;
CREATE TABLE IF NOT EXISTS Gender (
    Name            VARCHAR(100),
    CountryCode     CHAR(2),
    GenderLabel     VARCHAR(20),
-- got rid of this foreign key because gender is based on the first name and author name is a full name
--     FOREIGN KEY (Name) REFERENCES Author(Name),
    FOREIGN KEY (CountryCode) REFERENCES Countries(CountryCode),
    PRIMARY KEY (Name, CountryCode)
);
"""
with open("tables.txt", "w") as f:
    f.write(author_table)
    f.write(race_table)
    f.write(gender_table)
    f.write(journals_table)
    f.write(articles_table)
    f.write(country_table)
    f.write(institutions_table)